# Two Variable Linear Regression

## Overview

In this notebook I will be taking the data from two excel file "Simple one year academic data"  both AS and A2 and creating a data frame using the pandas library. I have deliberately split an excel sheet into two seperate files to explore putting them back together in python.  

As an academic exercise I will also be creating an SQL relation database with two "pages" and using the Student Id as a relational tag.

The data will be saved in the github so you can reproduce the results by changing the file name to a URL

After combining the data some basic analysis will be completed demonstrating several graph types.

Then a simple linear regression model will be run, splitting the data into learning and testing data.




## Step one: Load and clean

Load the data from the excel file and create two data frames "dfAS" and "dfA2" for the AS and A2 results.

### Tips and tricks:

Don't forget the file extension (i.e. .xlsx)

Also, for literal pathways sometimes you need a letter 'r' before the path

In [9]:
#this will keep all our graphs in the page
%matplotlib inline
# a few libraries that we will need
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# We need this library to read excel workbooks
import openpyxl 
#this allows you to open older versions of excel
import xlrd

# This should take the first sheet and use the first row as header and create a data frame
excel_file_as = (r"C:\Users\Mrs Farrelly\Documents\Academic data\Simple one year academic data AS results.xlsx")
df_file_as = pd.read_excel(excel_file_as, sheet_name = 0, header = 0)


excel_file_a2 = (r"C:\Users\Mrs Farrelly\Documents\Academic data\Simple one year academic data A2 results.xlsx")
df_file_a2 = pd.read_excel(excel_file_a2, sheet_name = 0, header = 0)

#Lets have a look at the first 5 rows and see if it worked
df_file_a2.head()

,Student Id,A2 Values
0,2598,589
1,2544,573
2,2592,573
3,2619,570
4,2582,565


In [28]:
df_file_as.head()

,Student Id,AS Values
0,2598,294
1,2544,295
2,2592,287
3,2619,278
4,2582,296


Now lets have a look at the data to make sure there aren't any missing values, unexpected 0's etc

In [16]:
print (df_file_as.dtypes)


Student Id    int64
AS Values     int64
dtype: object


In [36]:
df_file_as.describe()

,Student Id,AS Values
count,72.000000,72.000000
mean,2547.819444,234.375000
std,151.254969,49.195625
min,2409.000000,0.000000
25%,2484.250000,207.500000
50%,2540.500000,243.000000
75%,2581.250000,271.750000
max,3714.000000,296.000000


In [51]:
df_file_as.shape

(72, 2)

Hmmmm, a min value of 0 either means someone had a really bad day at the office or a missing result was passed as a 0. This is also way smaller than 2 times the STD below the mean so it would certainly come out as an outlier.

Also the column names having spaces in them are going to cause syntax issues.  Lets rename them with underscores.  Notice the inplace = True changes the original data frame.  I would say usually this is not a good idea and we should create new data frames when we make changes so we can track the changes.

In [39]:
df_file_as.rename(columns = {'AS Values':'AS_Values'}, inplace = True)
                    

WE can see here the new column name. 

In [40]:
df_file_as.head()

,Student Id,AS_Values
0,2598,294
1,2544,295
2,2592,287
3,2619,278
4,2582,296


Lets change the Student Id as well but this time lets create a new data frame

In [43]:
df_as_new = df_file_as.rename(columns = {'Student Id':'Student_Id'})

In [44]:
df_as_new.head()

,Student_Id,AS_Values
0,2598,294
1,2544,295
2,2592,287
3,2619,278
4,2582,296


Now lets go and change the A2 headers in the same way

In [46]:
df_a2_new = df_file_a2.rename(columns = {'Student Id':'Student_Id','A2 Values':'A2_Values'})
df_a2_new.head()

,Student_Id,A2_Values
0,2598,589
1,2544,573
2,2592,573
3,2619,570
4,2582,565


In [55]:
df_a2_new.shape

(72, 2)

Now let's go and have a look at that 0 value using the .query()

In [47]:
df_as_new.query("AS_Values == 0")

,Student_Id,AS_Values
14,3714,0


Looks like there is only "0" value so just getting rid of it won't cause too many problems with reducing the data size. Notice also that I've kept the name this time so I don't have to be so creative

In [52]:
df_as_new = df_as_new.drop(df_as_new.index[14])

In [53]:
df_as_new.describe()

,Student_Id,AS_Values
count,71.000000,71.000000
mean,2531.394366,237.676056
std,59.186263,40.730745
min,2409.000000,134.000000
25%,2483.500000,209.000000
50%,2537.000000,243.000000
75%,2580.500000,272.500000
max,2624.000000,296.000000


In [54]:
df_as_new.shape

(71, 2)

We can see that the number of columns is still 2 and the number of rows has reduced to 71.  This may create a problem when we combine our data sets so we will have to be careful to clean it up afterwards and match it using Student_Id. We can either combine at this stage or do some initial plots.  Let's go ahead and combine so we can use subplots in our comparisons.

# Step two: Combining

So let's use a concatonate statement to combine the two dataframes.  This seems ideal because the Student_Id's are in the same order. But wait I errased AS_Values row 14 when I was cleaning the data.

In [57]:
df_col_merged =pd.concat([df_as_new, df_a2_new], axis=1)
df_col_merged.head()

,Student_Id,AS_Values,Student_Id,A2_Values
0,2598.0,294.0,2598,589
1,2544.0,295.0,2544,573
2,2592.0,287.0,2592,573
3,2619.0,278.0,2619,570
4,2582.0,296.0,2582,565


In [58]:
df_col_merged.shape

(72, 4)

But wait I errased AS_Values row 14 when I was cleaning the data. Lets take a look at that column and see what has happened.


In [74]:
df_col_merged.loc[12:16]

,Student_Id,AS_Values,Student_Id,A2_Values
12,2478.0,264.0,2478,538
13,2537.0,277.0,2537,535
14,NaN,NaN,3714,534
15,2419.0,264.0,2419,532
16,2536.0,283.0,2536,528


We can see now that there is an NaN for AS_Values and the first Student_Id but the other row have matched up.  Let's drop the 14th row again and also get rid of the second Student_Id

In [76]:
df_cleaned = df_col_merged.drop(df_col_merged.index[14])
df_cleaned.loc[12:16]

,Student_Id,AS_Values,Student_Id,A2_Values
12,2478.0,264.0,2478,538
13,2537.0,277.0,2537,535
15,2419.0,264.0,2419,532
16,2536.0,283.0,2536,528


In [87]:
df_cleaned =df_cleaned.T.drop_duplicates().T
df_cleaned.loc[12:16]

,Student_Id,AS_Values,A2_Values
12,2478.0,264.0,538.0
13,2537.0,277.0,535.0
15,2419.0,264.0,532.0
16,2536.0,283.0,528.0


as we can see we still have all the corresponding values matching up but now we don't have Student_Id twice